In [1]:
#####Joint Embedding with Factorization Machinese#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id

In [3]:
###データの生成####
##データの設定
s = 3
k1 = 10
k2 = 10
k3 = 7
hh = 3500
item = 5000
element = 500
pt1 = np.random.poisson(30.0, hh)
pt2 = np.random.poisson(20.0, hh)
hhpt1 = np.sum(pt1)
hhpt2 = np.sum(pt2)
k_vec1 = np.repeat(1, k1)
k_vec2 = np.repeat(1, k2)
k_vec3 = np.repeat(1, k3)

In [4]:
##IDとインデックスの設定
#IDの設定
d_id1 = np.array(np.repeat(np.arange(hh), pt1), dtype="int16")
d_id2 = np.array(np.repeat(np.arange(hh), pt2), dtype="int16")
pt_id1 = np.array(list(itertools.chain(*[np.array(range(pt1[i]), dtype="int") for i in range(hh)])), dtype="int16")
pt_id2 = np.array(list(itertools.chain(*[np.array(range(pt2[i]), dtype="int") for i in range(hh)])), dtype="int16")

#インデックスの設定
d_list1 = [i for i in range(hh)]
d_list2 = [i for i in range(hh)]
for i in range(hh):
    d_list1[i] = np.array(np.where(d_id1==i)[0], dtype="int")
    d_list2[i] = np.array(np.where(d_id2==i)[0], dtype="int")

In [8]:
##レコードごとの要因とidをを生成
#レコードごとの頻度を生成
gamma0 = np.random.gamma(3.0, 1/0.5, hh)[d_id1]
freq = np.random.poisson(gamma0, hhpt1)
freq[freq < 1] = 1
N = np.sum(freq)

#greaterとlesserのインデックスを生成
flag1 = np.random.binomial(1, 0.5, N)
index_greater1 = np.array(np.where(flag1==1)[0], dtype="int")
index_lesser1 = np.array(np.where(flag1==0)[0], dtype="int")

#要因の長さに応じたidを生成
record_id = np.repeat(np.arange(hhpt1), freq)
record_id1 = record_id[index_greater1]
record_id2 = record_id[index_lesser1]
d_long = np.repeat(d_id1, freq)
d_long1 = d_long[index_greater1]
d_long2 = d_long[index_lesser1]

In [7]:
####シナリオを生成####



In [5]:




#トピックとアイテムの格納用配列
z11 = np.full((hhpt1, max_freq1), topic, dtype="int16")
z12 = np.full((hhpt1, max_freq2), topic, dtype="int16")
item_id11 = np.full((hhpt1, max_freq1), item, dtype="int16")
item_id12 = np.full((hhpt1, max_freq2), item, dtype="int16")
item_id2 = np.array(np.repeat(0, hhpt2), dtype="int16")

#多項分布からアイテムを生成
for i in range(hhpt1):
    index1 = np.arange(freq1[i])
    index2 = np.arange(freq2[i])
    z11[i, index1] = np.array(rmnom(theta0[d_id1[np.repeat(i, freq1[i])], ], freq1[i], topic, 0), dtype="int16")
    z12[i, index2] = np.array(rmnom(theta0[d_id1[np.repeat(i, freq2[i])], ], freq2[i], topic, 0), dtype="int16")
    item_id11[i, index1] = np.array(rmnom(phi_item[z11[i, index1], ], freq1[i], item, 0), dtype="int16")
    item_id12[i, index2] = np.array(rmnom(phi_item[z12[i, index2], ], freq2[i], item, 0), dtype="int16")
    
for i in range(hh):
    item_id2[d_list2[i]] = np.array(rmnom(phi_item[z2[d_list2[i]], ], pt2[i], item, 0), dtype="int16")

In [7]:
##エレメントを生成
#エレメントの格納用配列
element_id11 = np.full((hhpt1, max_freq1), item, dtype="int16")
element_id12 = np.full((hhpt1, max_freq2), item, dtype="int16")
element_id2 = np.array(np.repeat(0, hhpt2), dtype="int16")

#多項分布からエレメントを生成
for i in range(hhpt1):
    index1 = np.arange(freq1[i])
    index2 = np.arange(freq2[i])
    element_id11[i, index1] = np.array(rmnom(phi_element[z11[i, index1], ], freq1[i], element, 0), dtype="int16")
    element_id12[i, index2] = np.array(rmnom(phi_element[z12[i, index2], ], freq2[i], element, 0), dtype="int16")
    
for i in range(hh):
    element_id2[d_list2[i]] = np.array(rmnom(phi_element[z2[d_list2[i]], ], pt2[i], element, 0), dtype="int16")

In [ ]:
##レコード単位に拡張したidとインデックスを作成
#レコード単位のidを作成
N1 = np.sum(freq1)
N2 = np.sum(freq2)
N = N1 + N2
record_id1 = np.repeat(np.arange(hhpt1), freq1)
record_id2 = np.repeat(np.arange(hhpt1), freq2)


#拡張したidを作成
d_long1 = np.repeat(d_id1, freq1); d_long2 = np.repeat(d_id1, freq2)
item_long1 = item_id11[item_id11 < item]; item_long2 = item_id12[item_id12 < item]
element_long1 = element_id11[element_id11 < element]; element_long2 = element_id12[element_id12 < element]

#idのフラッグを作成
Zi1 = sparse.coo_matrix((np.repeat(1, N1), (np.arange(N1), record_id1)), shape=(N1, hhpt1)).tocsr()
Zi2 = sparse.coo_matrix((np.repeat(1, N2), (np.arange(N2), record_id2)), shape=(N2, hhpt1)).tocsr()

#レコードのインデックスを作成
record_list1 = [i for i in range(hhpt1)]
record_list2 = [i for i in range(hhpt1)]
record_index1 = [i for i in range(hhpt1)]
record_index2 = [i for i in range(hhpt1)]
for i in range(hhpt1):
    record_list1[i] = np.array(np.where(record_id1==i)[0], dtype="int")
    record_list2[i] = np.array(np.where(record_id2==i)[0], dtype="int")
    record_index1[i] = np.array(np.arange(freq1[i]), dtype="int8")
    record_index2[i] = np.array(np.arange(freq2[i]), dtype="int8")